In [2]:
import xml.etree.ElementTree as ET
import pandas as pd

xlm = ET.parse('test.xml')
tag_root = xlm.getroot()

# for tag_root in tag_root.getroot():#findall('DocumentProperties'):    
#     print(root_node)
#     tag2=root_node

    # for tag in root_node.findall('table'):    
    #     print(tag)
    #     # print(tag.count, tag)


# root_node = ET.parse('test.xml').f getroot() 
# print(root_node)

# for tag in root_node.findall('Cell'): 
#     print(tag)
    # value = tag.get(attribute) 
    # if value is not None: 
    #     print(value)
# Источник: https://pythononline.ru/osnovy/parsing-xml-python

In [19]:
# root_node.findtext('Row')
# for tag in main_tag.findall('DocumentProperties'):
#     print(tag)

# root_node.get('DocumentProperties')
# main_tag.getroot(tag='DocumentProperties')


for tag in tag_root:
    print(tag)

<Element '{urn:schemas-microsoft-com:office:office}DocumentProperties' at 0x00000268B90A8360>
<Element '{urn:schemas-microsoft-com:office:excel}ExcelWorkbook' at 0x00000268B90A8400>
<Element '{urn:schemas-microsoft-com:office:spreadsheet}Styles' at 0x00000268B90A8090>
<Element '{urn:schemas-microsoft-com:office:spreadsheet}Worksheet' at 0x00000268BABB4900>


In [80]:

level =   '{urn:schemas-microsoft-com:office:spreadsheet}Worksheet'
level += '/{urn:schemas-microsoft-com:office:spreadsheet}Table'
level += '/{urn:schemas-microsoft-com:office:spreadsheet}Row'

# df = pd.DataFrame({"Date":[], "Time":[], "Name":[], "Amount":[], "Balance":[]})

rows = {0:[], 1:[], 2:[], 3:[], 4:[], 5:[]}

flag1_mark = False  # Найти текст 'Карта'
flag2_mark = False  # Найти метку карты, найти идентификатор метки базе данных
flag3 = False  # Найти текст 'Карта'
flag_break = False
for i_row, tag_row in enumerate(tag_root.findall(level)):
    if i_row == 9:
        amounts = []
        for i, t1 in enumerate(tag_row.itertext()):
            if i >= 4: break            
            t2 = str.replace(t1, ' ', '')
            t3 = str.replace(t2, ',', '.')
            try: amount = float(t3)
            except: amount = None
            amounts.append(amount)
            
        print(amounts)
    elif i_row >= 19:        
        # print('row ', i_row, '-----------------', tag_row)

        for text in tag_row.itertext():

            print(text)



    # # print('---------------------------')
    # if flag2_mark:
    #     print('row ', i_row, '-----------------', tag_row)
    #     # for tag in tag_row:
    #         # print(tag, tag.text)
    #     for text in tag_row.itertext():
    #         print(text)
    # else:
    #     if flag1_mark:            
    #         # Найти метку карты, найти идентификатор метки базе данных
    #         # texts = tag_row.itertext()

    #         flag_break = True
    #         for i, text in  enumerate(tag_row.itertext()):
    #             # Получить ID из БД
    #             card_mark = text
    #             print(i, text)
    #             break
    #         flag2_mark = True
    #         flag_break = False

    #     else:
    #         for text in tag_row.itertext():
    #             if text == 'Карта':
    #                 flag1_mark = True
    #                 break
    # if flag_break:
    #     break


    # print(text)

#     print(tag, tag.text)
#     for tag2 in tag:
#         print('  - ', tag2, tag2.text)


[3432.97, 337.94, 129334.64, 126239.61]
row  19 ----------------- <Element '{urn:schemas-microsoft-com:office:spreadsheet}Row' at 0x000001B878ECE0E0>
2022-03-31
15:44
Перевод с карты
5 000,00
337.94
row  20 ----------------- <Element '{urn:schemas-microsoft-com:office:spreadsheet}Row' at 0x000001B878ECE720>
2022-03-31
202880
SBOL перевод 2202****6497 В. СЕРГЕЙ СЕРГЕЕВИЧ
row  21 ----------------- <Element '{urn:schemas-microsoft-com:office:spreadsheet}Row' at 0x000001B878ECEB80>
2022-03-31
15:44
Прочие операции
+1 000,00
5 337,94
row  22 ----------------- <Element '{urn:schemas-microsoft-com:office:spreadsheet}Row' at 0x000001B878EE12C0>
2022-03-31
262259
SBOL перевод 2202****6753 К. АЛЕКСАНДР ГЕННАДЬЕВИЧ
row  23 ----------------- <Element '{urn:schemas-microsoft-com:office:spreadsheet}Row' at 0x000001B878EE1720>
2022-03-30
02:44
Неизвестная категория(-)
199.00
4 337,94
row  24 ----------------- <Element '{urn:schemas-microsoft-com:office:spreadsheet}Row' at 0x000001B878EE1D60>
2022-03-

In [79]:

# df = pd.concat([df, ["Ddddate", "Tdddime", "Name", "Amount", "Balance"]])
# df
rows = {0:[], 1:[], 2:[], 3:[], 4:[], 5:[]}
# columns = ["Date", "Time", "Name", "Amount", "Balance"]
# df = pd.DataFrame(columns=columns)
# # pd.DataFrame(columns=["Date", "Time", "Name", "Amount", "Balance"])
# ser = pd.Series(data=columns, index=["dDate", "dTime", "dName", "dAmount", "dBalance"])
# df = pd.concat([df,ser], axis=1)
# df
# # df.append(ser, ignore_index=True)
rows

{0: [], 1: [], 2: [], 3: [], 4: [], 5: []}